# Домашнее задание 2

В этом задании:

1. Сделаем регрессию над данными через scikit-learn: сначала через регресию, потом через бустинг.
2. Сравним результаты с константным предсказанием.
3. Сделаем нейронную сеть на полносвязных слоях, обучим над теми же данными - и сравним с лин. регрессией и бустингом.



In [34]:
import pandas as pd
import numpy as np
import random

# Для воспроизводимости
seed = 0
np.random.seed(seed)
random.seed(seed)

In [35]:
df = pd.read_csv("insurance.csv")

### Задание №1:
Cделайте train/test split на данных в пропорции 0.2/0.8, залейте в лмс код, который в `df_train`, `df_test`
сохранит датафрейм с тренировочными и тестовыми данными соответственно.

P.S Использовать train_test_split из scikit-learn запрещено - разбивайте вручную через индексы.

In [3]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
int(df.shape[0] * 0.8)

1070

In [36]:
idx_test = np.random.choice(df.shape[0], size=int(0.2 * df.shape[0]), replace=False)
df_test = df.iloc[idx_test]
df_train = df.drop(index=idx_test)

### Задание №2:
Сделайте `OHE` на колонки sex, region, smoker.

Нужно сделать как на train, так и на test датасете.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.
При этом в `df_train`, `df_test` должны остаться старые колонки - т.е. его нужно обогатить.


In [37]:
df_train['sex'].value_counts()

sex
female    543
male      528
Name: count, dtype: int64

In [38]:
df_train['region'].value_counts()

region
southeast    294
southwest    273
northwest    254
northeast    250
Name: count, dtype: int64

In [39]:
df_train['smoker'].value_counts()

smoker
no     852
yes    219
Name: count, dtype: int64

In [40]:
df_train = pd.get_dummies(df_train, columns=['sex', 'smoker', 'region'], drop_first=False)
df_test = pd.get_dummies(df_test, columns=['sex', 'smoker', 'region'], drop_first=False)

In [41]:
print(df_train.columns)
print(df_test.columns)

Index(['age', 'bmi', 'children', 'charges', 'sex_female', 'sex_male',
       'smoker_no', 'smoker_yes', 'region_northeast', 'region_northwest',
       'region_southeast', 'region_southwest'],
      dtype='object')
Index(['age', 'bmi', 'children', 'charges', 'sex_female', 'sex_male',
       'smoker_no', 'smoker_yes', 'region_northeast', 'region_northwest',
       'region_southeast', 'region_southwest'],
      dtype='object')


### Задание №4:
Нормализуйте колонки, которые вы отметили в квизе.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.

Сдайте код, который модифицирует `df_train` и `df_test` так, чтобы численные колонки из прошлого пункта стали нормированы.

In [42]:
df_train.head()

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,True,False,False,True,False,False,False,True
2,28,33.000,3,4449.46200,False,True,True,False,False,False,True,False
3,33,22.705,0,21984.47061,False,True,True,False,False,True,False,False
4,32,28.880,0,3866.85520,False,True,True,False,False,True,False,False
6,46,33.440,1,8240.58960,True,False,True,False,False,False,True,False


In [43]:
# charges, bmi, age

for col in ['charges', 'bmi', 'age']:
    mean_value = df_train[col].mean()
    std_value = df_train[col].std()

    df_train[col] = (df_train[col] - mean_value)/std_value
    df_test[col] = (df_test[col] - mean_value)/std_value


In [44]:
y_train = df_train.pop("charges")
y_test = df_test.pop("charges")

### Задание №5
Реализуйте функцию, считающую `MSE` метрику.

Ваша функция должна уметь принимать torch.Tensor, numpy-массивы и pd.Series.

In [45]:
def metric(preds, y):
    return ((y - preds) ** 2).mean()

In [46]:
preds = np.array([0.21, 0.55, 0.95, 0.99, 0.45])
y = np.array([0.31, 0.04, 0.51, 0.24, 0.91])

assert metric(preds, y) == 0.24756

### Задание №6
Реализуйте бейзлайн на `LinearRegression` и `GradientBoostingRegressor`, отправьте метрики в ЛМС.

Используйте гиперпараметры по-умолчанию в обоих моделях.

In [47]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

gbr = GradientBoostingRegressor()
lr = LinearRegression()

gbr.fit(df_train, y_train)
lr.fit(df_train, y_train)

gbr_metric = metric(gbr.predict(df_test), y_test)
lr_metric = metric(lr.predict(df_test), y_test)

print(f"GBR MSE: {gbr_metric:.2f}\nLR MSE: {lr_metric:.2f}")

GBR MSE: 0.12
LR MSE: 0.22


In [48]:
# shorter alternative
for model in (LinearRegression(), GradientBoostingRegressor()):
    model.fit(df_train, y_train)
    preds = model.predict(df_test)
    score = metric(preds, y_test)
    print(model, score)

LinearRegression() 0.2228159571271519
GradientBoostingRegressor() 0.114906605210456


### Задание №7
Вычислите среднее значение целевой переменной на тренировочной выборке (train).

Подсчитайте MSE при константном предсказании этим средним и отправьте его в ЛМС.

In [49]:
np.repeat(y_train.mean(), len(y_test)).shape

(267,)

In [50]:
y_test.shape

(267,)

In [51]:
metric(np.repeat(y_train.mean(), len(y_test)), y_test)

1.1145089956822458

### Задание №8
Создайте сеть, состоящую из одного слоя Linear, залейте в лмс код, который в `model` запишет вашу модель

В качестве признаков используйте все колонки в текущем датасете, за исключением таргета

In [52]:
df_train.shape[1]

11

In [53]:
import torch.nn as nn


def build_model():
    model = nn.Linear(in_features=11, out_features=1)
    return model    

model = build_model()

In [54]:
import torch

torch.random.manual_seed(seed)
t_x_train = torch.from_numpy(df_train.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_train = torch.from_numpy(y_train.to_numpy()).to(dtype=torch.float32)

### Задание №9
Напишите функцию `train_loop`, которая будет учить модель по данным на 2к итераций.
Считайте, что данные уже хранятся в переменных `t_x_train`, `t_y_train`.
Ваша функция `train_loop` должна вернуть список из лоссов на каждой итерации (т.е. список длины 2000).

Используйте `learning_rate=1e-2` в оптимизаторе.

Для простоты за одну итерацию делайте проход вперед и проход назад на всех наших обучающих данных.
Это будет полный градиентный спуск (не по батчам) - можем себе позволить, данных немного.

_Подсказка 1_: Вам не обязательно учить модель на видеокарте, CPU будет достаточно.

_Подсказка 2_: `tqdm` - это библиотека, которая рисует прогресс итераций

In [55]:
import tqdm
from torch.optim.sgd import SGD


def train_loop(model: nn.Module) -> list[float]:
    losses = []
    optim = SGD(model.parameters(), lr=1e-2)
    criterion = nn.MSELoss()
    for _ in tqdm.trange(2000):
        optim.zero_grad()
        out = model(t_x_train)[:, 0]
        loss = criterion(out, t_y_train)
        loss.backward()
        losses.append(loss.item())
        optim.step()

    return losses

model = build_model()
losses = train_loop(model)

100%|████████████████████████████████████| 2000/2000 [00:00<00:00, 14029.27it/s]


In [56]:
len(losses)

2000

### Задание №10
Обучите модель, состоящую из одного слоя `Linear`.
Приложите в ЛМС метрику `MSE` на тестовых данных.
Используйте `learning_rate=1e-2` в оптимизаторе.

Когда будете тестировать, не забудьте перенести тестовые данные в `torch.Tensor`

In [57]:
t_x_test = torch.from_numpy(df_test.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_test = torch.from_numpy(y_test.to_numpy()).to(dtype=torch.float32)

model = build_model()
train_loop(model)
with torch.no_grad():
    y_pred = model(t_x_test)[:, 0]
metric(y_pred, t_y_test)

100%|████████████████████████████████████| 2000/2000 [00:00<00:00, 14083.61it/s]


tensor(0.2228)

In [58]:
nn.MSELoss()(y_pred, t_y_test)

tensor(0.2228)

### Задание №11
Вам необходимо усложнить существующую нейронную сеть, добавив один скрытый слой. 

Используйте следующие параметры: 

Размерность скрытого слоя: 6 нейронов, функция активации -  `ReLU`

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [61]:
import torch.nn as nn

def build_model():
    model = nn.Sequential()
    model.append(nn.Linear(11, 6))
    model.append(nn.ReLU())
    model.append(nn.Linear(6, 1))
    return model


model = build_model()

In [62]:
model

Sequential(
  (0): Linear(in_features=11, out_features=6, bias=True)
  (1): ReLU()
  (2): Linear(in_features=6, out_features=1, bias=True)
)

### Задание №12
Приложите в ЛМС метрику качества этой сети после 2к итераций обучения.
Эту модель можно обучить на CPU, не обязательно на видеокарте.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

In [63]:
t_x_test = torch.from_numpy(df_test.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_test = torch.from_numpy(y_test.to_numpy()).to(dtype=torch.float32)

model = build_model()
train_loop(model)
with torch.no_grad():
    y_pred = model(t_x_test)[:, 0]
metric(y_pred, t_y_test)

100%|█████████████████████████████████████| 2000/2000 [00:00<00:00, 8402.83it/s]


tensor(0.1380)

### Задание №13
Добавьте дополнительные слои в нейронную сеть

Вам необходимо усложнить нейронную сеть, добавив еще 2-3 скрытых слоя с такими же размерностями, как в предыдущем задании.

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [66]:
import torch.nn as nn

def build_model():
    model = nn.Sequential()
    model.append(nn.Linear(11, 6))
    model.append(nn.ReLU())
    model.append(nn.Linear(6, 6))
    model.append(nn.ReLU())
    model.append(nn.Linear(6, 1))
    return model

model = build_model()

### Задание №14
Приложите в ЛМС метрику качества после 2к итераций обучения.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

Эту модель можно обучить на CPU, не обязательно на видеокарте.

In [67]:
t_x_test = torch.from_numpy(df_test.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_test = torch.from_numpy(y_test.to_numpy()).to(dtype=torch.float32)

model = build_model()
train_loop(model)
with torch.no_grad():
    y_pred = model(t_x_test)[:, 0]
metric(y_pred, t_y_test)

100%|█████████████████████████████████████| 2000/2000 [00:00<00:00, 6712.26it/s]


tensor(0.1344)